# Spark & Jupyter | Ambiente de desenvolvimento e testes

In [ ]:
!pip install unidecode

**Objetivo:** contar a quantidade de ocorrências das palavras em um arquivo de texto.

## Passos

1. Escrever uma função para mapear as palavras: WordMapperClass -> wordMapper(); 
2. Escrever uma função para reduzir a lista de frequência: WordReducerClass -> wordReducer();
3. Escrever um script que aponte para as funções criadas e execute os métodos: WordCounterClass -> main();

In [2]:
import os 
import sys

os.environ["PYSPARK_SESSION"] = sys.executable

# Spark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('SparkHelloWord').getOrCreate()

### Passo 1: WordMapper Class

- A função irá rodar uma só vez por cada linha do arquivo de texto; 
- **Input:** {"num_da_linha": "Texto da linha"}
- **Output:** {"palavra": "quantidade"}

In [ ]:
def WordMapper(args):
    
    for line in sys.stdin:
        for word in line.split():
            print "%s\t%s" % (word,1)
    
    return par_palavras

## Passo 2: WordReducer Class

- A função irá rodar uma só vez por cada par de chave-valor ordenado pelo Hadoop;

- **Input:** {"palavra": "lista_de_frequencia"}
- **Output:** {"palavra": "contagem"}

In [ ]:
def WordReducer(args):
    
    (last_key,count) = (None, 0)
    for line in sys.stdin:
        (key, value) = line.strip().split("\t")
        if last_key and last_key!=key:
            print "%s\t%s" % (last_key,count)
            (last_key,count) = (key,int(value))
        else:
            (last_key,count) = (key, count + int(value))
    
    return 

## Passo 3: Main

- A função principal do programa contendo todas as configurações necessárias para realizar a tarefa;

In [ ]:
import sys

def WordCounter(input_path, output_path):
    return


In [2]:
import os # Sistema operacional 
import sys # Funções e variáveis da shell do Python
import getopt # Criação de opções de entrada pela CLI
import pandas as pd # Pandas
import pyspark # Spark 
from pyspark import SparkContext # SparkContext
from pyspark.sql import SparkSession # SparkSession
from pyspark.sql.functions import * # Funções SQL

# SparkSession
spark = SparkSession.builder.appName("NotebookTeste").getOrCreate()
# SparkContext
#sc = SparkContext()

### Extraindo os dados

In [3]:
caminho_texto = "./README.md"
caminho_csv = "./nomes.csv"
#nomes = spark.read.format("csv").load(caminho_csv)

## Job 1 | Contagem de palavras

### Leitura do arquivo

In [5]:
def ler_arquivo(caminho):
    try:
        with open(caminho, "r") as arquivo:
            return list(enumerate(map(str.strip, arquivo), start=1))
    except FileNotFoundError:
        return []


## Transformações 

### Funções e Classes

#### WordMapper Class

- A função irá rodar uma só vez por cada linha do arquivo de texto; 
- **Input:** {"num_da_linha": "Texto da linha"}
- **Output:** {"palavra": "quantidade"}

In [6]:
# import sys

class WordMapper:
    def __init__(self):
        pass

    def map(self, line):
        words = line.split()
        for word in words:
            print(f"{word}\t1")

# Exemplo de uso
mapper = WordMapper()
for line in sys.stdin:
    mapper.map(line)

#### WordReducer Class

- A função irá rodar uma só vez por cada par de chave-valor ordenado pelo Hadoop;
- **Input:** {"palavra": "lista_de_frequencia"}
- **Output:** {"palavra": "contagem"}

In [7]:
# import sys

class WordReducer:
    def __init__(self):
        self.last_key = None
        self.count = 0

    def reduce(self, key, value):
        if self.last_key and self.last_key != key:
            print(f"{self.last_key}\t{self.count}")
            self.last_key, self.count = key, int(value)
        else:
            self.last_key, self.count = key, self.count + int(value)

    def finalize(self):
        if self.last_key:
            print(f"{self.last_key}\t{self.count}")

# Exemplo de uso
reducer = WordReducer()
for line in sys.stdin:
    key, value = line.strip().split("\t")
    reducer.reduce(key, value)

reducer.finalize()


#### Limpeza das linhas

In [8]:
def processar_linha(linha):
    mapeamento = {}
    for palavra in [
        re.sub(r'[^a-z]', '', unidecode(w.lower()))
        for w in re.findall(r'\b\w+\b', unidecode(linha.lower()))
    ]:
        if palavra:
            mapeamento[palavra] = mapeamento.get(palavra, 0) + 1
    return mapeamento

### Armazenamento de dados em um DataFrame

#### Migrando o conteúdo do arquivo de texto para um RDD

In [1]:
# Ler o arquivo | SparkContext (RDD)
linhas = sc.textFile(caminho_texto)
#linhas.collect()
# Limpeza das linhas
palavras = linhas.flatMap(lambda linha: linha.strip().lower.split())
# Contagem das palavras
contagem = palavras.map(lambda palavra: (palavra, 1))
# Redução das quantidades
reduzido = contagem.reduceByKey(lambda a,b: a + b)
dados = reduzido
colunas = ["Palavra", "Quantidade"]
df = spark.createDataFrame(dados, colunas)

NameError: name 'sc' is not defined

In [3]:
# Importe as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType
import re
from unidecode import unidecode

# Inicie uma sessão Spark
spark = SparkSession.builder.appName("ProcessarArquivo").getOrCreate()

# Defina uma função para processar as linhas e criar um dicionário
def processar_linha(linha):
    mapeamento = {}
    for palavra in [
        re.sub(r'[^a-z]', '', unidecode(w.lower()))
        for w in re.findall(r'\b\w+\b', unidecode(linha.lower()))
    ]:
        if palavra:
            mapeamento[palavra] = mapeamento.get(palavra, 0) + 1
    return mapeamento

# Registre a função como uma função UDF (User-Defined Function)
schema = StructType([StructField("linha", StringType(), True)])
processar_linha_udf = udf(processar_linha, schema)

# Carregue o arquivo de texto em um DataFrame
caminho_texto = "./README.md"  # Substitua pelo caminho do seu arquivo
df = spark.read.text(caminho_texto)

# Aplique a função UDF para processar as linhas e criar uma coluna com o resultado
df_com_mapeamento = df.withColumn("mapeamento", processar_linha_udf(df["value"]))

# Colete o DataFrame em uma lista de dicionários
lista_de_dicionarios = df_com_mapeamento.select("mapeamento").rdd.flatMap(lambda x: x).collect()

# Crie um dicionário final combinando todos os dicionários na lista
dicionario_final = {}
for dicionario in lista_de_dicionarios:
    for chave, valor in dicionario.items():
        dicionario_final[chave] = dicionario_final.get(chave, 0) + valor

# Exiba o dicionário final
print(dicionario_final)

# Encerre a sessão Spark
spark.stop()


AttributeError: items

#### Armazenamento de dados em um DataFrame | Pandas

In [ ]:
# Criar um DataFrame do Spark
colunas = ["Palavra", "Quantidade"]
df = spark.createDataFrame(mapeamento_palavras, colunas)


## Job 2 | AWS Glue
- [] Alterar os valores da coluna `nome` para maiúsculo;
- [] Contagem das linhas presentes no DataFrame;
- [] Contagem de nomes agrupados por `ano` e `sexo`, ordenados por `ano` decrescente;
- [] Nome feminino mais registrado e o respectivo `ano`;
- [] Nome masculino mais registrado e o respectivo `ano`;
- [] Total de registros para cada `ano`. Apresentar as 10 primeiras linhas, ordenados por `ano` crescente;

### Transformações | Job 2

## Job 3 | Coleta in Batch

- [] importar a lib boto3
- [] passar as credenciais ao container `spark_jupyter`
- [] ler 2 arquivos em .csv sem filtrar os dados
- [] carregar os dados para o S3
- [] acessar a AWS e grava no S3, no bucket definido com RAW Zone

>      - no momento da gravação dos dados deve-se considerar o padrão: <nome do bucket>\<camada de armazenamento>\<origem do dado>\<formato do dado>\<especificação do dado>\<data de processamento separada por ano\mes\dia>\<arquivo>
>
>            Por exemplo:
>
>                   S3:\\data-lake-do-fulano\Raw\Local\CSV\Movies\2022\05\02\movies.csv
>
>                   S3:\\data-lake-do-fulano\Raw\Local\CSV\Series\2022\05\02\series.csv
>

- [] criar um container Docker com volume para (i) armazenar os arquivos.csv e (ii) executar o job.py

### Transformações | Job 3

### Escrevendo o resultado

In [ ]:
caminho_saida = "./outputs"
resultado.write.format("csv").save(caminho_saida)

### Finalizando a sessão

In [ ]:
spark.stop()